In [ ]:
# when using google colab, uncomment the following lines:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r /content/drive/MyDrive/Uva/data/outputs/ /content/
# !pip install -r /content/drive/MyDrive/Uva/requirements.txt
# !pip install --force-reinstall --no-deps  git+https://github.com/neelnanda-io/TransformerLens/

In [ ]:
from transformer_lens import HookedTransformer
from huggingface_hub import login
from src.utils.file_utils import load_files_named, filter_adv_org_samples
import pandas as pd

login(token='')

In [1]:

output_folder = 'work/outputs'

adv_sample_1 = load_files_named(output_folder, file_starts_with='adv_sample_1_3',if_gpu=False)
org_sample_1 = load_files_named(output_folder, file_starts_with='org_sample_1_3',if_gpu=False)

adv_sample = load_files_named(output_folder, file_starts_with='adv_sample_3',if_gpu=False)
org_sample = load_files_named(output_folder, file_starts_with='org_sample_3',if_gpu=False)

In [2]:
len(adv_sample_1), len(org_sample_1), len(adv_sample), len(org_sample)

(27, 27, 24, 24)

In [7]:
unique_adv_1, unique_org_1  = filter_adv_org_samples(adv_sample_1, org_sample_1)
unique_adv, unique_org = filter_adv_org_samples(adv_sample, org_sample)

In [ ]:
unique_adv.shape, unique_org.shape, unique_adv_1.shape, unique_org_1.shape

In [ ]:
model = HookedTransformer.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    #default_prepend_bos = False
    # refactor_factored_attn_matrices=True
)

In [10]:
cols = ['org_str','adv_str', 'org_label', 'adv_label']

org_str = model.to_string(unique_org)
org_str += model.to_string(unique_org_1)
adv_str = model.to_string(unique_adv)
adv_str += model.to_string(unique_adv_1)

org_label = ['false'] * len(unique_org) + ['true'] * len(unique_org_1)
adv_label = ['true'] * len(unique_adv) + ['false'] * len(unique_adv_1)

df = pd.DataFrame(list(zip(org_str, adv_str, org_label, adv_label)), columns=cols)

In [ ]:
df.to_csv('data/circuit_identification_data/adv_bias/corrupt_adv_bias_eap_0.csv', index=False)